<a href="https://colab.research.google.com/github/mini3713/mini3713/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%2B%EB%94%A5%EB%9F%AC%EB%8B%9D/XGBoost_05_XGBoost_Unveiled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5 XGBoost 소개**
=================


5.1 XGBoost 모델 만들기 
====================


> ### 붓꽃 데이터셋 - 분류

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=2)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', max_depth=6, learning_rate=0.1, n_estimators=100, n_jobs=-1)
# booster는 기본학습기, gbtree는 그레이디언트 부스팅, 다른학습기는 8장에서
# objective는 목적함수, 이 함수를 통해 출력된 값이 최소가 되는 방향으로 학습 
# objective는 이진분류일때 binary:logistic, 다중분류는 multi:softmax(클래스 레이블 반환), softprob(클래스 확률 반환)
# learning_rate는 지정된 비율로 각 트리의 가중치를 감소시켜 분산을 억제
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('점수: ' + str(score))

점수: 0.9736842105263158


> ### 당뇨병 데이터셋 - 회귀

In [ ]:
# 데이터셋으로부터 입력값과 타깃값을 생성
X, y = datasets.load_diabetes(return_X_y=True)

In [ ]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror', max_depth=6, learning_rate=0.1, n_estimators=100, n_jobs=-1)
# 목적함수는 MSE 평균제곱오차
scores = cross_val_score(xgb, X, y, scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE 평균: %0.3f' % (rmse.mean()))

RMSE: [63.033 59.689 64.538 63.699 64.661]
RMSE 평균: 63.124


In [ ]:
pd.DataFrame(y).describe()
# rmse가 1 표준편차 이내이므로 괜찮은 결과라고 할 수 있다

,0
count,442.000000
mean,152.133484
std,77.093005
min,25.000000
25%,87.000000
50%,140.500000
75%,211.500000
max,346.000000


5.2 힉스 보손 찾기 - 사례 연구
====================
여기서 개발할 모델은 경연 당시에 XGBoost 개발팀에서 제공한 기본모델이다  
사이킷런의 XGBoost모델이 개발되기 전이므로, 개발 당시의 모델과 함수를 사용하였다

In [11]:
df = pd.read_csv('/content/atlas-higgs-challenge-2014-v2.csv', nrows=250000)
df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label,KaggleSet,KaggleWeight
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2.150,0.444,46.062,1.24,-2.475,113.497,0.000814,s,t,0.002653
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,0.725,1.158,-999.000,-999.00,-999.000,46.226,0.681042,b,t,2.233584
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,2.053,-2.028,-999.000,-999.00,-999.000,44.251,0.715742,b,t,2.347389
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,1.660654,b,t,5.446378
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,1.904263,b,t,6.245333


In [12]:
# 캐글 훈련데이터와 같게 만드는 작업
del df['Weight']
del df['KaggleSet']
df = df.rename(columns={"KaggleWeight":"Weight"})

In [13]:
# Label 열을 마지막으로 옮기는 작업
label_col = df['Label']
del df['Label']
df['Label'] = label_col

In [14]:
df.head()

,EventId,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight,Label
0,100000,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,...,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497,0.002653,s
1,100001,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,...,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226,2.233584,b
2,100002,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,...,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251,2.347389,b
3,100003,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,-0.000,5.446378,b
4,100004,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,...,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000,6.245333,b


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   EventId                      250000 non-null  int64  
 1   DER_mass_MMC                 250000 non-null  float64
 2   DER_mass_transverse_met_lep  250000 non-null  float64
 3   DER_mass_vis                 250000 non-null  float64
 4   DER_pt_h                     250000 non-null  float64
 5   DER_deltaeta_jet_jet         250000 non-null  float64
 6   DER_mass_jet_jet             250000 non-null  float64
 7   DER_prodeta_jet_jet          250000 non-null  float64
 8   DER_deltar_tau_lep           250000 non-null  float64
 9   DER_pt_tot                   250000 non-null  float64
 10  DER_sum_pt                   250000 non-null  float64
 11  DER_pt_ratio_lep_tau         250000 non-null  float64
 12  DER_met_phi_centrality       250000 non-null  float64
 13 

In [16]:
df['Label'].replace(('s', 'b'),(1,0), inplace=True)

In [17]:
X = df.iloc[:,1:31]
y = df.iloc[:,-1]

> ### 측정 지표
>
> $$ \sqrt{2((s+b+ b_{reg})ln(1+ \frac{s}{b+ b_{reg} } )-s)} $$
>
> * $s$는 진짜 양성 비율, $b$는 가짜 양성 비율, $b_{reg}$는 규제상수 항으로 10

> ### 가중치
> * 실제 상황에서는 $b$(배경잡음)가 $s$(신호)보다 훨씬 많으므로 가중치를 다르게 부여해서 불균형을 보상할 수 있다
> * 예로 신호가 배경잡음보다 1000배 드물게 나타난다고 하면, $s = 1, b = 1/1000$으로 준다
> * 이 가중치를 테스트 세트의 신호와 배경잡음수에 맞게 스케일을 조정해 주어야 한다
> * 테스트세트는 550,000개, 훈련세트의 길이는(len(y)) 250,000이다
> * 가중치에 증가된 샘플 비율을 곱하면 테스트 세트에 맞게 가중치를 조정할 수 있다

In [20]:
df['test_Weight'] = df['Weight'] * 550000 / len(y)

In [21]:
# 신호와 배경잡음에 대한 가중치를 합하여서 모델에서 사용할 변수를 생성 
s = np.sum(df[df['Label']==1]['test_Weight'])
b = np.sum(df[df['Label']==0]['test_Weight'])

> ### 모델

In [23]:
import xgboost as xgb
# 사이킷런이 아닌 DMatrix로 XGBoost 모델 초기화
# DMatrix를 사용하면 dataframe 형식을 numpy로 변환해주지 않아도, numpy 형식의 입력데이터가 생성됨 
xgmat = xgb.DMatrix(X, y, missing = -999.0, weight=df['test_weight'])

In [25]:
# 빈 param 딕셔너리를 초기화
param = {}
# 목적함수를 아래와같이 설정하는 것은 시그모이드 함수를 적용하기전의 점수를 최적화한다는 의미 
# 이 목적함수는 이진분류 모델에 사용 
param['objective'] = 'binary:logitraw'

In [26]:
# 양성샘플인 신호샘플의 수가 훨씬 적기 때문에 샘플의 불균형을 보상해 준다 
# 이는 테스트세트에서 더 나은 성능을 내는데 도움이 된다 
param['scale_pos_weight'] = b/s
# 학습률은 0.1
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
# 매개변수를 리스트로 만들고, 평가지표 상위 15%를 신호로 판단함
plst = list(param.items())+[('eval_metric', 'ams@0.15')]
# 트리가 추가됨에 따라 점수를 볼 수 있도록 watchlist 생성
watchlist = [(xgmat, 'train')]
num_round = 120

In [27]:
print('데이터 로딩 완료, 트리 부스팅 시작')
bst = xgb.train(plst, xgmat, num_round, watchlist)
bst.save_model('higgs.model')
print('훈련종료')

데이터 로딩 완료, 트리 부스팅 시작
[0]	train-auc:0.910911	train-ams@0.15:3.69957
[1]	train-auc:0.915308	train-ams@0.15:3.97123
[2]	train-auc:0.917743	train-ams@0.15:4.06746
[3]	train-auc:0.919345	train-ams@0.15:4.20976
[4]	train-auc:0.920139	train-ams@0.15:4.13
[5]	train-auc:0.921023	train-ams@0.15:4.1755
[6]	train-auc:0.921944	train-ams@0.15:4.26182
[7]	train-auc:0.922337	train-ams@0.15:4.26232
[8]	train-auc:0.92333	train-ams@0.15:4.32669
[9]	train-auc:0.924186	train-ams@0.15:4.38384
[10]	train-auc:0.924737	train-ams@0.15:4.38879
[11]	train-auc:0.925317	train-ams@0.15:4.4079
[12]	train-auc:0.925923	train-ams@0.15:4.44648
[13]	train-auc:0.926334	train-ams@0.15:4.45051
[14]	train-auc:0.926958	train-ams@0.15:4.48914
[15]	train-auc:0.927428	train-ams@0.15:4.51754
[16]	train-auc:0.928081	train-ams@0.15:4.53708
[17]	train-auc:0.928468	train-ams@0.15:4.5494
[18]	train-auc:0.929035	train-ams@0.15:4.59646
[19]	train-auc:0.92952	train-ams@0.15:4.6434
[20]	train-auc:0.93004	train-ams@0.15:4.7141
[21]	train-au